In [ ]:
import os

from cmdstanpy.utils import _TMPDIR, cmdstan_path, set_cmdstan_path
from cmdstanpy.model import CmdStanModel
_TMPDIR

In [ ]:
cmdstan_path()

In [ ]:
# set_cmdstan_path(os.path.expanduser(os.path.join('~', 'github', 'stan-dev', 'cmdstan')))

In [ ]:
bernoulli_dir = os.path.join(cmdstan_path(), 'examples', 'bernoulli')
bernoulli_path = os.path.join(bernoulli_dir, 'bernoulli.stan')

bernoulli_model = CmdStanModel(stan_file=bernoulli_path)
print(bernoulli_model)

bern_json = os.path.join(bernoulli_dir, 'bernoulli.data.json')

bern_data = { "N" : 10, "y" : [0,1,0,0,0,0,0,0,0,1] }

The `CmdStanModel` method `sample` runs the NUTS-HMC sampler and returns a `CmdStanMCMC` object.  By default, `sample` runs 4 sampler chains.

In [ ]:
bern_fit = bernoulli_model.sample(data=bern_data, save_warmup=True, iter_warmup=200)

In [ ]:
bern_fit.draws().shape

In [ ]:
bern_fit.draws(inc_warmup=True, concat_chains=True).shape

In [ ]:
bernoulli_ppc_model = CmdStanModel(stan_file='bernoulli_ppc.stan')


In [ ]:
bernoulli_ppc_model.generate_quantities(data=bern_data, fitted_params='bernoulli-fitted-params.csv')

In [ ]:
print(bern_fit.runset)
' '.join(bern_fit.runset.cmds[0])

In [ ]:
print(bern_fit)

### Access the sample: the `CmdStanMCMC` object attributes and methods

#### Summarize the results

The `CmdStanMCMC` method `summary` returns output of CmdStan bin/stansummary as pandas.DataFrame.  The `summary` report provides estimates of both the parameter value and the goodness of fit.

In [ ]:
bern_fit.summary()

#### Run sampler diagnostics

The `CmdStanMCMC` method `diagnose` prints the output of CmdStan bin/stansummary to the console.  This is useful when the summary report shows `R_hat` values outside of the range of .99 to 1.01, or the number of effective samples (`N_eff` value) is below 5% of the total draws in the sample. 

In [ ]:
bern_fit.diagnose()

In [ ]:
bern_draws_pd = bern_fit.draws_pd()

In [ ]:
bern_draws_pd

In [ ]:
bern_draws_pd.shape, bern_draws_pd.columns

In [ ]:
thetas = bern_fit.stan_variable(name='theta')
thetas.shape
thetas.plot.density()

In [ ]:
bern_fit.draws().shape

In [ ]:
bern_fit.column_names[7], bern_fit.draws()[0:3,0,7]

#### Get HMC sampler tuning parameters

##### stepsize

The `CmdStanMCMC` property `stepsize` property is a 1-D numpy ndarray which contains the stepsize used by the sampler for each chain.  This array is created at the same time as the `sample` and `metric` arrays are created.

At the end of adaptation, the stepsize for the 4 chains in this example is:

In [ ]:
bern_fit.stepsize

#### metric_type, metric

The `metric` property is an numpy ndarray which contains the metric used by the sampler for each chain.  This array is created at the same time as the `sample` and `stepsize` arrays are created.

At the end of adaptation, the metric for the 4 chains in this example is:

In [ ]:
bern_fit.metric_type,  bern_fit.metric